In [1]:
import numpy as np
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator


F:\Ancondainstall\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
x_train = np.load('x_train.npy')
print(x_train.shape)

(57504, 32, 32, 1)


In [3]:
y_train = np.load('y_train.npy')
print(y_train.shape)

(57504, 43505)


In [4]:
x_test = np.load('x_test.npy')
print(x_test.shape)

(2722, 32, 32, 1)


In [5]:
y_test = np.load('y_test.npy')
print(y_test.shape)

(2722, 43505)


In [6]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (32,32,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

In [8]:
model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(43505, activation = "softmax"))

In [9]:
x_train.shape

(57504, 32, 32, 1)

In [10]:
y_train.shape


(57504, 43505)

In [9]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [10]:
epochs = 10 
batch_size = 100

In [11]:
# With data augmentation to prevent overfitting (accuracy 0.99286)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 2, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
 - 1549s - loss: 10.3247 - acc: 0.0367 - val_loss: 7.2380 - val_acc: 0.1389
Epoch 2/10
 - 3788s - loss: 10.1202 - acc: 0.0373 - val_loss: 7.1889 - val_acc: 0.1389
Epoch 3/10
 - 2429s - loss: 10.0665 - acc: 0.0372 - val_loss: 6.7691 - val_acc: 0.1389
Epoch 4/10
 - 1660s - loss: 10.0307 - acc: 0.0377 - val_loss: 6.7285 - val_acc: 0.1389

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 5/10
 - 6010s - loss: 9.9884 - acc: 0.0377 - val_loss: 6.6861 - val_acc: 0.1389
Epoch 6/10
 - 1418s - loss: 9.9823 - acc: 0.0372 - val_loss: 6.7093 - val_acc: 0.1389
Epoch 7/10
